## 8.1 Triple Step

In [95]:
// naive recursive algorithm
function stepCount(n) {
    return n <= 0 ? 0 : _stepCount(n, n, 0)
}

function _stepCount(n, current, ways) {
    if(current === 0) {
        ways++;
        return ways;
    }
    else if (current < 0) {
        return ways;
    }
    else {
        ways = _stepCount(n, current - 1, ways);
        ways = _stepCount(n, current - 2, ways);
        ways = _stepCount(n, current - 3, ways);
    }
    return ways;
}

stepCount(10)

274

In [109]:
// memoized version
function stepMemo(n) {
    let memo = {};
    return n <= 0 ? 0 : _stepMemo(n, n, 0, memo)
}

function _stepMemo(n, current, ways, memo) {
    if(memo[current]) {
        return ways + memo[current];
    }
    else if(current === 0) {
        ways++;
        return ways;
    }
    else if (current < 0) {
        return ways;
    }
    else {
        ways = _stepMemo(n, current - 1, ways, memo);
        ways = _stepMemo(n, current - 2, ways, memo);
        ways = _stepMemo(n, current - 3, ways, memo);
        memo[current] = ways;
    }
    return ways;
}

console.log(stepMemo(10))
console.log(stepMemo(100))
console.log(stepMemo(1000))

274
1.803963808151009e+26
2.7588428077664853e+264


* exactly the same as __bottom-up Fibonacci__
* only difference is that it keeps track of 3 values instead of 2

In [100]:
// bottom up
function stepDP(n) {
    if(n <= 0) {
        return 0;
    }
    else if (n <= 2) {
        return n;
    }
    else {
        let oneStep = 2;
        let twoStep = 1;
        let threeStep = 1;
        for(let i = 3; i < n; i++) {
            let temp = oneStep + twoStep + threeStep;
            threeStep = twoStep;
            twoStep = oneStep;
            oneStep = temp;
        }
        return oneStep + twoStep + threeStep;
    }
}

console.log(stepDP(10))
console.log(stepDP(100))
console.log(stepDP(1000))

274
1.803963808151009e+26
2.7588428077664853e+264


### Book Solutions
* naive rec. alg: O(3$^{n}$)
    - similar to fibonacci
* memoized solution: O(n)
    - b/c the only time you have a recursive call is when you have to figure out the value of a new number
    - else, it'll be in the memo table already

In [110]:
// simple implementation

function countWays(n) {
    if (n < 0) {
        return 0;
    }
    // assume that countWays(0) = 1
    else if (n === 0) {
        return 1;
    }
    else {
        return countWays(n - 1) + countWays(n - 2) + countWays(n - 3);
    }
}

countWays(10)

274

In [112]:
// memoized

function countWaysMemo(n) {
    let memo = [];
    return _countWaysMemo(n, memo);
}

function _countWaysMemo(n, memo) {
    if (n < 0) {
        return 0;
    }
    else if (n === 0) {
        return 1;
    }
    else if (memo[n]) {
        return memo[n]
    }
    else {
        memo[n] = _countWaysMemo(n - 1, memo) + _countWaysMemo(n - 2, memo) + _countWaysMemo(n - 3, memo);
        return memo[n];
    }
}

countWaysMemo(100)

1.803963808151009e+26

## 8.2 Robot in Grid

In [67]:
function robo(grid, r = 0, c = 0, path = '') {
    if(r === 0 && c === 0) {
        return true;
    }
    else if (r >= 0 && c >= 0) {
        path = robo(grid, r - 1, c, path);
        path = robo(grid, r, c - 1, path);
    }
    return path;
}

var grid = [
    [ 0,  0, 0, 0, 0],
    [ 0,  0, 0, 0,'x'],
    ['x', 0, 0, 0, 0],
    [ 0,  0,'x',0, 0],
    ['x', 0, 0, 0, 0],
];

console.log(robo(grid, grid.length - 1, grid[0].length) - 1);

0


### Book Solutions:
* solution: __O(2$^{r + c}$), where r = # rows, and c = # cols__
    1. if maze is empty or null, return null
    2. call on _getPath(maze, r, c, path)
        - path is an array of points (r, c)
    3. first checks if row or cols are out of bounds, then checks if the spot can be stepped on
    4. if it can, then check if it is at the origin
    5. if it is not, move down or right by 1 point
    6. once it is at the origin, push the current (r, c) point to the path array
    7. return true
* overlapping problem:
    - for a path to [r, c], we check [r - 1, c] or [r, c - 1]
    - then we check the adjacent cells of those two so [r - 2, c], [r - 1, c - 1], [r - 1, c - 1], [r, c - 2]
    - as you can see, we check [r - 1, c - 1] twice so that is unnecessary work that can be avoided and instead memoized
* memoized solution: __O(r * c) b/c only hit each cell once__
    1. same as before but we have a hash table called failedPoints
    2. if the cell is a dead-end and we have to backtrack, add the rc into failedPoints
    3. and if we have already arrived at the cell before, i.e. failedPoints[rc] === true, then we return false and don't check that spot again
***
* different from my solution b/c I used a string to get the answer instead when I should've used an array
    - it would've made more sense to push a point to an array than to just add 'R' or 'L' every time I went in a certain direction
    - not to mention, it is easier to retrieve the answer since it passes a reference of the array and not the value so once the algo is done, all changes that are made to the path array by the rec. algo are then saved and not overwritten

In [95]:
function getPath(maze) {
    if(maze == null || maze.length === 0) {
        return null;
    }
    let path = [];
    if(_getPath(maze, maze.length - 1, maze[0].length - 1, path)) {
        return path;
    }
    return null;
}

function _getPath(maze, r, c, path) {
    if(c < 0 || r < 0 || !maze[r][c]) {
        return false;
    }
    
    let isAtOrigin = (r === 0) && (c === 0);
    
    // this helps check if there even IS a path
    // b/c if it is not at the origin and both _getPath() calls return false,
    // we know there is no way for a path to form
    if(isAtOrigin || _getPath(maze, r, c - 1, path) || _getPath(maze, r - 1, c, path)) {
        path.push({r, c});
        return true;
    }
    return false;
}

var maze = [
    [1,1,1,1,1],
    [1,1,1,0,1],
    [1,0,1,1,1],
    [1,1,0,1,1],
    [0,1,1,0,1]
];

getPath(maze);

[
  { r: 0, c: 0 },
  { r: 1, c: 0 },
  { r: 1, c: 1 },
  { r: 1, c: 2 },
  { r: 2, c: 2 },
  { r: 2, c: 3 },
  { r: 3, c: 3 },
  { r: 3, c: 4 },
  { r: 4, c: 4 }
]

In [107]:
function getPathDP(maze) {
    if(maze == null || maze.length === 0) {
        return null;
    }
    let path = [];
    let failedPoints = {};
    if(_getPathDP(maze, maze.length - 1, maze[0].length - 1, path, failedPoints)) {
        return path;
    }
    return null;
}

function _getPathDP(maze, r, c, path, failedPoints) {
    if(c < 0 || r < 0 || !maze[r][c]) {
        return false;
    }
    
    // if we have already visited this point
    if(failedPoints[`${r}${c}`]) {
        return false;
    }
    
    let isAtOrigin = (r === 0) && (c === 0);
    
    if(isAtOrigin || _getPathDP(maze, r, c - 1, path, failedPoints) || _getPathDP(maze, r - 1, c, path, failedPoints)) {
        path.push({r, c});
        return true;
    }
    failedPoints[`${r}${c}`] = true;
    return false;
}

var maze = [
    [1,1,1,1,1],
    [1,1,1,0,1],
    [1,0,1,1,0],
    [1,0,0,1,1],
    [0,1,1,0,1]
];

getPathDP(maze);

[
  { r: 0, c: 0 },
  { r: 1, c: 0 },
  { r: 1, c: 1 },
  { r: 1, c: 2 },
  { r: 2, c: 2 },
  { r: 2, c: 3 },
  { r: 3, c: 3 },
  { r: 3, c: 4 },
  { r: 4, c: 4 }
]

## 8.3 Magic Index

In [109]:
// O(n)
function magicIndex(arr) {
    let found = false;
    for(let i = 0; i < arr.length; i++) {
        if(arr[i] === i) {
            found = true;
            break;
        }
    }
    return found;
}

var arr = [0, 3, 5, 7, 9, 10, 11, 12, 14, 20];
magicIndex(arr)

true

In [143]:
// function binarySearch(arr, num, low, high) {
//     let midpoint = Math.trunc( low + ( (high - low) / 2) );
//     if(num === arr[midpoint]) {
//         return true;
//     }
//     else if (high < low) {
//         return false;
//     }
//     else if (num < arr[midpoint]) {
//         return binarySearch(arr, num, low, midpoint - 1);
//     }
//     else if (num > arr[midpoint]) {
//         return binarySearch(arr, num, midpoint + 1, high);
//     }
// }

// var arr = [0, 3, 5, 7, 9, 10, 11, 12, 14, 20];
// console.log(binarySearch(arr, 8, 0, arr.length))


// O(log n) solution based off of binary search
function binaryMI(arr, low, high) {
    let midpoint = Math.trunc( low + ( (high - low) / 2) );
    if(arr[midpoint] === midpoint) {
        return true;
    }
    else if (high < low) {
        return false;
    }
    // if arr[i] > i, so if arr[9] > 9,
    // we know that any numbers below it would have at
    // least one number that is a magic index
    else if (arr[midpoint] > midpoint) {
        return binaryMI(arr, low, midpoint - 1);
    }
    // else if arr[i] < i, then we know that any numbers larger than it
    // would yield a magic index
    else {
        return binaryMI(arr, midpoint + 1, high);
    }
}

// true
var arr = [0, 3, 5, 7, 9, 10, 11, 12, 14, 20];
console.log(binaryMI(arr, 0, arr.length));

// false
arr = [1, 3, 5, 7, 9, 10, 11, 12, 14, 20];
console.log(binaryMI(arr, 0, arr.length));

// true w/ non-distinct values
arr = [-10, -5, 2, 2, 2, 3, 4, 7, 9, 12, 13];
console.log(binaryMI(arr, 0, arr.length));

// true w/ sample from book
arr = [-40, -20, -1, 1, 2, 3, 5, 7, 9, 12, 13];
console.log(binaryMI(arr, 0, arr.length));

true
false
true
true


### Book Solutions:
* __SINCE THEY ASKED ABOUT SORTED ARRAYS AND A FOLLOW-UP ON NON-SORTED ARRAYS, YOU SHOULD ALWAYS TRY TO SEE IF IT IS RELATED TO BINARY SEARCH SOMEHOW__
* brute force algorithm: 
    - loop through the entire array and see if it fulfills the condition
* binary search method:
    - exactly the same as my solution
    - when A[mid] < mid, we know that the magic index cannot be on the left side and must be on the right side
    - this is b/c if A[mid] is already small, then moving from i --> i - 1 would have the values also decrease by 1+
    - since the array is sorted as well, these values will always be small
    - this applies if A[mid] > mid too but all the values on the right side, i --> i + 1, would be too big
* will this work with non-distinct values? __No__
    - we can't really conclude which side the magic index could be on
    - thus, we should compare midIndex and midValue (value that is at the middle of the entire array) for equality first, then:
        - left: search from start --> Math.min(midIndex - 1, midValue)
        - right: search from Math.max(midIndex + 1, midValue) --> end

In [146]:
function magicFast(array) {
    return _magicFast(array, 0, array.length - 1);
}

function _magicFast(array, start, end) {
    if(end < start) {
        return -1;
    }
    let mid = Math.trunc((start + end) / 2);
    if(array[mid] === mid) {
        return mid;
    }
    else if (array[mid] > mid) {
        return _magicFast(array, start, mid - 1);
    }
    else {
        return _magicFast(array, mid + 1, end);
    }
}

// true
var arr = [0, 3, 5, 7, 9, 10, 11, 12, 14, 20];
console.log(magicFast(arr));

// false
arr = [1, 3, 5, 7, 9, 10, 11, 12, 14, 20];
console.log(magicFast(arr));

// true w/ non-distinct values
// I DON'T KNOW WHY IT WORKS BUT IT SHOULDN'T
arr = [-10, -5, 2, 2, 2, 3, 4, 7, 9, 12, 13];
console.log(magicFast(arr));

// true w/ sample from book
arr = [-40, -20, -1, 1, 2, 3, 5, 7, 9, 12, 13];
console.log(magicFast(arr));

0
-1
7
7


In [ ]:
// to make it work with non-distinct elements

function magicFast(array) {
    return _magicFast(array, 0, array.length - 1);
}

function _magicFast(array, start, end) {
    if(end < start) {
        return -1;
    }
    
    let midIndex = Math.trunc((start + end) / 2);
    let midValue = array[midIndex];
    if(midValue === midIndex) {
        return midIndex;
    }
    
    // search left
    let leftIndex = Math.min(midIndex - 1, midValue);
    let left = _magicFast(array, start, leftIndex);
    if(left >= 0) {
        return left;
    }
    
    // search right
    let rightIndex = Math.max(midIndex + 1, midValue);
    let right = _magicFast(array, rightIndex, end);
    
    return right;
}

## 8.4 Power Set

In [43]:
function powerSet(set) {
    let pSet = [];
    let max = 1 << set.length;
    
    for(let i = 0; i < max; i++) {
        let subset = convertInt(i, set);
        pSet.push(subset);
    }
    return pSet;
}

function convertInt(num, set) {
    let subset = []
    for(let i = 0; i < set.length; i++) {
        // this creates a mask that looks at the bit at the
        // ith position
        let mask = 1 << i;
        // then if the bit at the ith position in our num
        // is set, then that means that the element 
        // in set[i] is present, so we add that to the subset
        if( (num & mask) > 0 ) {
            subset.push(set[i]);
        }
    }
    return subset
}

powerSet(['a', 'b', 'c'])

[
  [],
  [ 'a' ],
  [ 'b' ],
  [ 'a', 'b' ],
  [ 'c' ],
  [ 'a', 'c' ],
  [ 'b', 'c' ],
  [ 'a', 'b', 'c' ]
]

### Book Solutions:
* there are 2$^{n}$ subsets for a set b/c when creating a set, there are 2 options for each element in that set
    - it can either be in the set or not in the set, so 2 possibilities
    - thus for each element, there are 2 choices for it so it can be expressed as {2 * 2 * 2 ... * 2} for every element
* time/space complexity = O(n * 2$^{n}$)
    - this is b/c we are returning a list of subsets and each element will be contained in half of the subsets
    - so it would be n (number of elements) * number of subsets
* recursion solution:
    1. compute the first set
    2. then clone the results
    3. then add the next element in the set to each of the cloned sets

In [39]:
function returnSubsets(set) {
    let subsets = [];
    const recurse = function(currSet, remainingSet) {
        subsets.push(currSet);
        for(let i = 0; i < remainingSet.length; i++) {
            recurse(currSet.concat([remainingSet[i]]), remainingSet.slice(i + 1));
        }
    };
    recurse([], set);
    return subsets;
}

console.log(returnSubsets(['a', 'b', 'c']))

[
  [],
  [ 'a' ],
  [ 'a', 'b' ],
  [ 'a', 'b', 'c' ],
  [ 'a', 'c' ],
  [ 'b' ],
  [ 'b', 'c' ],
  [ 'c' ]
]


## 8.5 Recursive Multiply

In [47]:
// naive rec. alg

function recMult(a, b) {
    let max = Math.max(a,b);
    let min = Math.min(a,b);
    return _recMult(max, min);
}

function _recMult(max, min) {
    if(min === 0) {
        return 0;
    }
    else {
        return max + _recMult(max, min - 1);
    }
}

recMult(13, 9);

117

In [107]:
function largestPower(n) {
    let x;
    while(n !== 0) {
        x = n;
        n &= (n - 1);
    }
    return x;
}


function figureExponent(n) {
    let exp = 0;
    while(n > 0) {
        n >>= 1;
        if(n !== 1) {
            exp++;  
        }
    }
    return exp;
}

function optMult(a, b) {
    let max = Math.max(a, b);
    let min = Math.min(a, b);
    let lp = largestPower(min);
    let exp = figureExponent(lp);
    let remainder = min - lp;
    return (max << exp) + _optMult(max, remainder);
}

function _optMult(max, min) {
    if(min === 0) {
        return 0;
    }
    else {
        return max + _optMult(max, min - 1);
    }
}

console.log(optMult(13, 9));

117


### Book Solutions
* what does it mean to multiply?
    - 8 x 7 = 8 + 8 + 8 + 8 + 8 + 8 + 8
* a way to think about 8 x 7 is to count the number of cells in an 8x7 matrix
    - a fast way to do it is to count a 4 x 7 grid and then double it
    - and we can count a 4 x 7 grid by counting a 2 x 7 grid and doubling that
* naive rec. solution:
    1. figure out the bigger and the smaller number
    2. smaller number >> 2
    3. compute the half-product
        - if it is uneven, we compute the other half
        - if it is not, we double it
    4. then we return the additions of the halves
***
* flaw with the naive solutin b/c there is duplicated work
    - suppose we have 17 x 23
    - the two halves would be (8 x 23) + (9, 23)
    - for both, the respective halves are:
        - (8 x 23) = (4 x 23) + (4 x 23)
        - (9 x 23) = (4 x 23) + (5 x 23)
   - therefore, you can memoize the solution to one of these and you would be able to speed it up
***
* memoized solution:
    1. declare a new array called memo
    2. same conditionals and an extra conditional to return memo[smaller] if it is present
    3. then once we have computed the two halves, we can put it in the memo table
        - memo[smaller] = side1 + side2
***
* with the memoized solution, even numbers are much faster than odd numbers to multiply
    - this is b/c when we do 30 x 35, we can do 2 * (15 x 35)
    - but with 31 x 35, we have to do (15 x 35) + (16 x 35), then we continue to halve it
    - this is unnecessary
* instead, what we can do is 2 * (15 x 35) + 35
* so the logic is to divide the smaller number by 2 and double the result
    - and if the smaller number is odd, we just add the bigger number to the result
***
* fastest solution:
    1. divide smaller number by 2
    2. if smaller number is even: halfProd + halfProd
    3. if smaller number is odd: halfProd + halfProd + bigger
* for example: 3 x 3
    1. 3 >> 2 = 1
    2. since it is odd, we do (3 + 3 + 3) = 9

In [123]:
// naive solution

function minProduct(a, b) {
    let bigger = a < b ? b : a;
    let smaller = a < b ? a : b;
    return minProductHelper(smaller, bigger);
}

function minProductHelper(smaller, bigger) {
    if(smaller === 0) {
        return 0;
    }
    else if (smaller === 1) {
        return bigger;
    }
    
    let s = smaller >> 1; // divide by 2
    let side1 = minProduct(s, bigger);
    let side2 = side1;
    if(smaller % 2 === 1) {
        side2 = minProductHelper(smaller - s, bigger);
    }
    
    console.log({side1, side2, total: side1 + side2})
    return side1 + side2;
}

minProduct(13, 9)

{ side1: 13, side2: 13, total: 26 }
{ side1: 26, side2: 26, total: 52 }
{ side1: 13, side2: 13, total: 26 }
{ side1: 13, side2: 13, total: 26 }
{ side1: 13, side2: 26, total: 39 }
{ side1: 26, side2: 39, total: 65 }
{ side1: 52, side2: 65, total: 117 }


117

In [115]:
// memoized solution

function minProductMemo(a, b) {
    let bigger = a < b ? b : a;
    let smaller = a < b ? a : b;
    let memo = [];
    return minProductHelperMemo(smaller, bigger, memo);
}

function minProductHelperMemo(smaller, bigger, memo) {
    if(smaller === 0) {
        return 0;
    }
    else if (smaller === 1) {
        return bigger;
    }
    else if (memo[smaller]) {
        return memo[smaller];
    }
    
    let s = smaller >> 1; // divide by 2
    let side1 = minProductMemo(s, bigger, memo);
    let side2 = side1;
    if(smaller % 2 === 1) {
        side2 = minProductHelperMemo(smaller - s, bigger, memo);
    }
    
    memo[smaller] = side1 + side2;
    return memo[smaller];
}

minProductMemo(13, 9)

117

In [117]:
// faster solution

// naive solution

function minProduct2(a, b) {
    let bigger = a < b ? b : a;
    let smaller = a < b ? a : b;
    return minProductHelper2(smaller, bigger);
}

function minProductHelper2(smaller, bigger) {
    if(smaller === 0) {
        return 0;
    }
    else if (smaller === 1) {
        return bigger;
    }
    
    let s = smaller >> 1;
    let halfProd = minProductHelper2(s, bigger);
    
    if(smaller % 2 === 0) {
        return halfProd + halfProd;
    }
    else {
        return halfProd + halfProd + bigger;
    }
}

minProduct2(13, 9)

117

## 8.6 Towers of Hanoi

In [9]:
class Towers {
    constructor(n) {
        this.towers = [ [], [], [] ];
    }
    
    solve(n) {
        for(let i = n; i > 0; i--) {
            this.towers[0].push(i);
        }
        console.log({before: this.towers})
        this.moveTo(n, this.towers[0], this.towers[1], this.towers[2]);
    }
    
    moveTo(n, origin, buffer, destination) {
        if(n > 0) {
            this.moveTo(n - 1, origin, destination, buffer);
            this.moveTop(origin, destination);
            this.moveTo(n - 1, buffer, origin, destination);
        }
    }
    
    moveTop(origin, destination) {
        destination.push(origin.pop());
    }
}

var hanoi = new Towers();
hanoi.solve(4);
console.log(hanoi.towers)

{ before: [ [ 4, 3, 2, 1 ], [], [] ] }
[ [], [], [ 4, 3, 2, 1 ] ]


### Book Solutions:
* __BASE CASE AND BUILD APPROACH__
* cases:
    - n = 1: move Disk 1 from Tower 1 to Tower 3
    - n = 2: 
        1. move D1: T1 --> T2
        2. move D2: T1 --> T3
        3. move D1 from T2 --> T3
    - n = 3:
        1. move top 2: T1 --> T2
        2. move D3: T1 --> T3
        3. move top 2: T2 --> T3
    - n = 3:
        1. move top 3: T1 --> T2
        2. move D4: T1 --> T3
        3. move top 3: T2 --> T3
* essentially, you know that you can move (n - 1) disks to a buffer, then move the last disk to the last tower, then move the rest to the last tower
***
pseudo code:

moveDisks(n, origin, destination buffer)

    if (n <= 0) return
    
        // move top n - 1 disks from tower 1 to tower 2
        moveDisks(n - 1, origin, buffer, destination)
        
        // then move the nth disk from the tower 1 to tower 3
        moveTop(origin, destination)
        
        // then move the top n - 1 disks from tower 2 to tower 3
        moveDisks(n - 1, buffer, destination origin)

## 8.7 Permutations without Dups: